In [1]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer                                                                                                                                                                 
from sklearn.preprocessing import OneHotEncoder, StandardScaler                                                                                                                                               
from sklearn.pipeline import Pipeline                                                                                                                                                                         
from sklearn.impute import SimpleImputer

from sklearn.model_selection import KFold, cross_val_score

from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer

RANDOM = 123
np.random.seed(RANDOM)

import time
import warnings
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv("../data/CW1_train.csv")
X = train.drop(columns=["outcome"])
y = train["outcome"]

categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()

cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM)

baseline_model = GradientBoostingRegressor(n_estimators=200, random_state=RANDOM)

# Test Approaches: dropping redundant features vs. creating composite features

In [4]:
#Baseline 
                                                                                                                                                                    
preprocessor_baseline = ColumnTransformer([                                                                                                                                                                   
    ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols),                                                                                                                              
    ('num', 'passthrough', numerical_cols)                                                                                                                                                                      
])       

pipeline_baseline = Pipeline([("prep", preprocessor_baseline), ("model", baseline_model)])
scores_baseline = cross_val_score(pipeline_baseline, X, y, cv=cv, scoring='r2', n_jobs=-1)
print(f'Baseline (all features): {scores_baseline.mean():.4f} ± {scores_baseline.std():.4f}')

Baseline (all features): 0.4686 ± 0.0175


In [5]:
# Drop Redundant Features
drop_cols = ['price', 'x', 'y', 'z']
numerical_cols_reduced = [col for col in numerical_cols if col not in drop_cols]

preprocessor_reduced = ColumnTransformer([                                                                                                                                                                   
    ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols),                                                                                                                              
    ('num', 'passthrough', numerical_cols_reduced)                                                                                                                                                                      
])
pipeline_reduced = Pipeline([("prep", preprocessor_reduced), ("model", baseline_model)])
scores_reduced = cross_val_score(pipeline_reduced, X, y, cv=cv, scoring='r2', n_jobs=-1)
print(f'Reduced Features (dropped {drop_cols}): {scores_reduced.mean():.4f} ± {scores_reduced.std():.4f}')

Reduced Features (dropped ['price', 'x', 'y', 'z']): 0.4700 ± 0.0175


In [6]:
# Composite Features
X_composite = X.copy()  
X_composite['volume'] = X['x'] * X['y'] * X['z']
X_composite["price_per_carat"] = X["price"] / (X["carat"] + 1e-6)

drop_cols_composite = ['price', 'x', 'y', 'z']
numeric_composite = [col for col in X_composite.columns                                                                                                                                                       
                       if col not in categorical_cols and col not in drop_cols_composite]    

preprocessor_composite = ColumnTransformer([                                                                                                                                                                   
    ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols),                                                                                                                              
    ('num', 'passthrough', numeric_composite)                                                                                                                                                                      
])

pipeline_composite = Pipeline([("prep", preprocessor_composite), ("model", baseline_model)])
scores_composite = cross_val_score(pipeline_composite, X_composite, y, cv=cv, scoring='r2', n_jobs=-1)
print(f'Composite Features (added volume & price_per_carat, dropped {drop_cols_composite}): {scores_composite.mean():.4f} ± {scores_composite.std():.4f}')

Composite Features (added volume & price_per_carat, dropped ['price', 'x', 'y', 'z']): 0.4688 ± 0.0165


In [7]:
# Summary
print("\nMULTICOLLINEARITY HANDLING STRATEGY COMPARISON:")
print(f"Baseline (all features):      {scores_baseline.mean():.4f} ± {scores_baseline.std():.4f}")                                                                                                            
print(f"Reduced (drop redundant):     {scores_reduced.mean():.4f} ± {scores_reduced.std():.4f}")                                                                                                              
print(f"Composite (volume, ppc):      {scores_composite.mean():.4f} ± {scores_composite.std():.4f}") 



MULTICOLLINEARITY HANDLING STRATEGY COMPARISON:
Baseline (all features):      0.4686 ± 0.0175
Reduced (drop redundant):     0.4700 ± 0.0175
Composite (volume, ppc):      0.4688 ± 0.0165


Takeaways:                                                                                                                                                                                                    
  - Dropping redundant features (price, x, y, z) gives a small but consistent improvement                                                                                                                       
  - Composite features didn't add value - the model already captures the relationships                                                                                                                          
  - Tree-based models handle multicollinearity well, but less noise still helps                                                                                                                                 
                                                                                                                                                                                                                
  Decision: Use the reduced feature set going forward.                     

# Latent Feature Interactions

In [8]:
X_reduced = X.drop(columns=['price', 'x', 'y', 'z'])

# Latent Feature Groups
latent_uniform = ['a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5']                                                                                                                                 
latent_gaussian = ['a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']  

X_interactions = X_reduced.copy()

# Cross group interactions
for i in range(6, 11):
    X_interactions[f'a{i}_x_b{i}'] = X_reduced[f'a{i}'] * X_reduced[f'b{i}']     

# Within group interactions
X_interactions['a6_x_a7'] = X_reduced['a6'] * X_reduced['a7']                                                                                                                                                 
X_interactions['b6_x_b7'] = X_reduced['b6'] * X_reduced['b7'] 

numeric_interactions = [col for col in X_interactions.columns if col not in categorical_cols]

preprocessor_interactions = ColumnTransformer([                                                                                                                                                                   
    ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols),                                                                                                                              
    ('num', 'passthrough', numeric_interactions)                                                                                                                                                                      
])

pipeline_interactions = Pipeline([("prep", preprocessor_interactions), ("model", baseline_model)])
scores_interactions = cross_val_score(pipeline_interactions, X_interactions, y, cv=cv, scoring='r2', n_jobs=-1)
print(f'Latent Feature Interactions: {scores_interactions.mean():.4f} ± {scores_interactions.std():.4f}')
print(f"vs Reduced baseline: {scores_reduced.mean():.4f} ± {scores_reduced.std():.4f}")


Latent Feature Interactions: 0.4703 ± 0.0185
vs Reduced baseline: 0.4700 ± 0.0175


  Takeaway: 
  Essentially no improvement. The increase in std (0.0185 vs 0.0175) suggests added noise rather than signal. GradientBoosting already captures these interactions internally through its tree splits.
                                                                                                                                                                                                                
  Decision: Skip explicit interactions - not worth the added complexity.

# Polynomial Features

In [9]:
from sklearn.preprocessing import PolynomialFeatures                                                                                                                                                          
                                                                                                                                                                                                            
                                                                                                                                                                              
X_reduced = X.drop(columns=['price', 'x', 'y', 'z'])                                                                                                                                                          
                                                                                                                                                                                                            
# Separate latent Gaussian for polynomial expansion                                                                                                                                                           
latent_gaussian = ['a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']                                                                                                                              
other_numeric = [col for col in X_reduced.columns                                                                                                                                                             
                if col not in categorical_cols and col not in latent_gaussian]                                                                                                                               
                                                                                                                                                                                                            
# Preprocessor with polynomial features on Gaussian latents only                                                                                                                                              
preprocessor_poly = ColumnTransformer([                                                                                                                                                                       
    ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols),                                                                                                                              
    ('poly', PolynomialFeatures(degree=2, include_bias=False), latent_gaussian),                                                                                                                              
    ('num', 'passthrough', other_numeric)                                                                                                                                                                     
])                                                                                                                                                                                                            
                                                                                                                                                                                                            
pipe_poly = Pipeline([('prep', preprocessor_poly), ('model', baseline_model)])                                                                                                                                
scores_poly = cross_val_score(pipe_poly, X_reduced, y, cv=cv, scoring='r2', n_jobs=-1)                                                                                                                        
print(f"Polynomial (degree=2) on Gaussian latents: {scores_poly.mean():.4f} ± {scores_poly.std():.4f}")                                                                                                       
print(f"vs Reduced baseline: {scores_reduced.mean():.4f} ± {scores_reduced.std():.4f}")         

Polynomial (degree=2) on Gaussian latents: 0.4645 ± 0.0179
vs Reduced baseline: 0.4700 ± 0.0175


  Takeaway: Polynomial features hurt performance. The expansion (10 features → 65 features with degree 2) likely added noise and caused the model to overfit on spurious patterns. Tree-based models already    
  capture nonlinearities through splits.                                                                                                                                                                        
                                                                                                                                                                                                                
  Decision: Skip polynomial features.    

# Target Transformation
EDA showed heteroskedasticity (errors increase for extreme values). A target transformation might stabilize variance and improve predictions.      

In [10]:
from sklearn.compose import TransformedTargetRegressor

X_reduced = X.drop(columns=['price', 'x', 'y', 'z'])
numeric_reduced = [col for col in X_reduced.columns if col not in categorical_cols]

preprocessor_reduced = ColumnTransformer([                                                                                                                                                                   
    ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols),                                                                                                                              
    ('num', 'passthrough', numeric_reduced)                                                                                                                                                                      
])

# Since target has negative values, we can't use log directly                                                                                                                                                 
# Option 1: Shift + log                                                                                                                                                                                       
y_min = y.min()                                                                                                                                                                                               
shift = abs(y_min) + 1  # Ensure all values positive       

def shift_log(y):                                                                                                                                                                                             
      return np.log(y + shift)                                                                                                                                                                                  
                                                                                                                                                                                                            
def shift_exp(y):                                                                                                                                                                                             
    return np.exp(y) - shift                                                                                                                                                                                  
                                                                                                                                                                                                            
# Option 2: sqrt with sign preservation                                                                                                                                                                       
def signed_sqrt(y):                                                                                                                                                                                           
    return np.sign(y) * np.sqrt(np.abs(y))                                                                                                                                                                    
                                                                                                                                                                                                            
def signed_square(y):                                                                                                                                                                                         
    return np.sign(y) * (y ** 2)

# Choose transformation
# Test 1: No transformation (baseline)                                                                                                                                                                        
pipe_no_transform = Pipeline([('prep', preprocessor_reduced), ('model', baseline_model)])                                                                                                                     
scores_no_transform = cross_val_score(pipe_no_transform, X_reduced, y, cv=cv, scoring='r2', n_jobs=-1)                                                                                                        
                                                                                                                                                                                                            
# Test 2: Shifted log transformation                                                                                                                                                                          
model_log = TransformedTargetRegressor(                                                                                                                                                                       
    regressor=Pipeline([('prep', preprocessor_reduced), ('model', baseline_model)]),                                                                                                                          
    func=shift_log,                                                                                                                                                                                           
    inverse_func=shift_exp                                                                                                                                                                                    
)                                                                                                                                                                                                             
scores_log = cross_val_score(model_log, X_reduced, y, cv=cv, scoring='r2', n_jobs=-1)                                                                                                                         
                                                                                                                                                                                                            
# Test 3: Signed sqrt transformation                                                                                                                                                                          
model_sqrt = TransformedTargetRegressor(                                                                                                                                                                      
    regressor=Pipeline([('prep', preprocessor_reduced), ('model', baseline_model)]),                                                                                                                          
    func=signed_sqrt,                                                                                                                                                                                         
    inverse_func=signed_square                                                                                                                                                                                
)                                                                                                                                                                                                             
scores_sqrt = cross_val_score(model_sqrt, X_reduced, y, cv=cv, scoring='r2', n_jobs=-1)                                                                                                                       
                                                                                                                                                                                                            
print("TARGET TRANSFORMATION RESULTS:")                                                                                                                                                                       
print(f"No transformation:  {scores_no_transform.mean():.4f} ± {scores_no_transform.std():.4f}")                                                                                                              
print(f"Shifted log:        {scores_log.mean():.4f} ± {scores_log.std():.4f}")                                                                                                                                
print(f"Signed sqrt:        {scores_sqrt.mean():.4f} ± {scores_sqrt.std():.4f}")             


TARGET TRANSFORMATION RESULTS:
No transformation:  0.4700 ± 0.0175
Shifted log:        0.4524 ± 0.0173
Signed sqrt:        0.4145 ± 0.0112


  Takeaway: Target transformations significantly hurt performance. The model handles the original target distribution well. Transformations distort the relationships GradientBoosting was learning effectively.
                                                                                                                                                                                                                
  Decision: Keep target untransformed. 

  Key insight: Feature engineering provided minimal gains. GradientBoosting already handles multicollinearity, interactions, and nonlinearity through its tree structure. Only removing redundant features      
  helped slightly.                                                                                                                                                                                              
                                                                                                                                                                                                                
  Best config entering Phase 5:                                                                                                                                                                                 
  - Model: GradientBoostingRegressor                                                                                                                                                                            
  - Features: Reduced set (drop price, x, y, z)                                                                                                                                                                 
  - CV R²: 0.4700 ± 0.0175    

In [1]:
from sklearn.preprocessing import PolynomialFeatures                                                                                                         
import numpy as np            

In [ ]:
latent_uniform = ['a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5']                                                                                
latent_gaussian = ['a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']                                                                             
                                                                                                                                                            
# Create engineered features                                                                                                                                 
X_eng = X.copy()                                                                                                                                             
                                                                                                                                                            
# 1. Interactions within latent groups (a*b pairs)                                                                                                           
for i in range(1, 11):                                                                                                                                       
    X_eng[f'ab_{i}'] = X[f'a{i}'] * X[f'b{i}']                                                                                                               
                                                                                                                                                            
# 2. Sum/mean aggregations                                                                                                                                   
X_eng['a_sum'] = X[[f'a{i}' for i in range(1,11)]].sum(axis=1)                                                                                               
X_eng['b_sum'] = X[[f'b{i}' for i in range(1,11)]].sum(axis=1)                                                                                               
X_eng['ab_diff'] = X_eng['a_sum'] - X_eng['b_sum']                                                                                                           
                                                                                                                                                            
# 3. Gaussian latent squared terms (capture non-linearity)                                                                                                   
for col in latent_gaussian:                                                                                                                                  
    X_eng[f'{col}_sq'] = X[col] ** 2                                                                                                                         
                                                                                                                                                            
print(f"Original features: {X.shape[1]}")                                                                                                                    
print(f"Engineered features: {X_eng.shape[1]}")                                                                                                              
                                                                                                                                                            
# Update column lists                                                                                                                                        
cat_cols_eng = cat_cols                                                                                                                                      
num_cols_eng = [c for c in X_eng.columns if c not in cat_cols]                                                                                               
                                                                                                                                                            
# New preprocessor                                                                                                                                           
preprocessor_eng = ColumnTransformer([                                                                                                                       
    ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols_eng),                                                                                 
    ('num', 'passthrough', num_cols_eng)                                                                                                                     
])                                                                                                                                                           
                                                                                                                                                            
# Test with LightGBM (fastest)                                                                                                                               
pipe_lgb_eng = Pipeline([                                                                                                                                    
    ('prep', preprocessor_eng),                                                                                                                              
    ('model', LGBMRegressor(**search_lgb.best_params_, random_state=RANDOM, n_jobs=-1, verbose=-1))                                                          
])                                                                                                                                                           
                                                                                                                                                            
# Remove 'model__' prefix from params                                                                                                                        
lgb_params = {k.replace('model__', ''): v for k, v in search_lgb.best_params_.items()}                                                                       
pipe_lgb_eng = Pipeline([                                                                                                                                    
    ('prep', preprocessor_eng),                                                                                                                              
    ('model', LGBMRegressor(**lgb_params, random_state=RANDOM, n_jobs=-1, verbose=-1))                                                                       
])                                                                                                                                                           
                                                                                                                                                            
print("\nEvaluating LightGBM with engineered features...")                                                                                                   
start = time.time()                                                                                                                                          
scores = cross_val_score(pipe_lgb_eng, X_eng, y, cv=cv, scoring='r2', n_jobs=-1)                                                                             
elapsed = time.time() - start                                                                                                                                
                                                                                                                                                            
print(f"Completed in {elapsed:.1f}s")                                                                                                                        
print(f"LightGBM + Engineered CV R²: {scores.mean():.4f} ± {scores.std():.4f}")                                                                              
print(f"vs baseline LightGBM: 0.4753")           